In [1]:
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import numpy.linalg as linalg
import cv2
import time
assert cv2.__version__ == "3.2.0"
import random
orb = cv2.ORB_create()
N = 100
s = 8

In [3]:
#function for computing the fundamental matrix
def computeF(xl, xr):
    A = []
    for i in xrange(len(xl)):
        x = xl[i]
        x_prime = xr[i]
        #Put a row into the matrix, with formula fropm class.
        A.append([x[0]*x_prime[0], x[1]*x_prime[0], x_prime[0],
                  x[0]*x_prime[1], x[1]*x_prime[1], x_prime[1], x[0], x[1], 1])
    #Get actual fundamental matrix.
    A = np.array(A)
    _, _, V = linalg.svd(A)
    F = V[-1,:].reshape((3,3))
    U, s, V = linalg.svd(F)
    sigma = np.diag(s)
    sigma[2,2] = 0
    return np.matmul(U, np.matmul(sigma, V))
    
def ransacF(points, kp1, kp2):
    n = 0 
    currF = None
    maxInliers = 0
    # run ransac
    for n in xrange(N):
        sampledMatches = random.sample(points, s)
        leftPoints = [kp1[point.queryIdx] for point in sample]
        rightPoints = [kp2[point.trainIdx] for point in sample]
        # compute our F matrix
        FToTest = computeF(leftPoints, rightPoints)
        numIn = 0
        for match in points:
            p = kp1[match.queryIdx].pt
            q = kp1[match.trainIdx].pt
            if (abs(np.matmul([p[0], p[1], 1], np.matmul(FToTest, [q[0], q[1], 1])))) < .01:
                numIn += 1
        if numIn > maxInliers:
            maxInliers = numIn
            currF = FToTest
    return currF

In [ ]:
def updateLoc(F):
    # TODO

In [ ]:
cap = cv2.VideoCapture('motion.mp4')
i = 0
last = None
lastDes = None
lastIm = None
while cap.isOpened():
    i += 1
    
    ret, frame = cap.read()
    if i % 10 != 0:
        continue
    if ret == False:
        break
    grayImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp, des = orb.detectAndCompute(grayImg, None)
    
    if last == None:
        last = kp
        lastDes = des
        lastIm = grayImg
        continue
    
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(lastDes, des, k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m)
    F = ransacF(good, lastDes, des)
    updateLoc(F)
    last = kp
    lastDes = des
    
    
